In [2]:
import pandas as pd
import numpy as np
import re
from homologaciones.exportadores import exportadores_dict, importadores_dict

In [3]:
imports = pd.read_csv('matched_imports.csv')

In [3]:
ruta_medicamentos = 'C:/Users/juan-/Downloads/pharma/Bases/20220516_MEDICAMENTOS ONCOLOGICOS BIOLOGICOS_PW_AP_PW.xlsx'
MedicamentosDB = pd.read_excel(ruta_medicamentos, sheet_name='BIOLOGICOS')

## Importadores y Exportadores Homologación.

In [24]:
importadores = imports['Importador'].value_counts()
importadores = sorted(list(dict(importadores).keys()))
len(importadores)

200

In [ ]:
exportadores = imports['Exportador'].value_counts()
exportadores = sorted(list(dict(exportadores).keys()))
exportadores

## funciones

### funciones de normalizacion y homologacion

In [4]:
def homologar_importador(importador: str) -> str:
    """"""
    try:
        importador_h = importadores_dict[importador] if importador != 'sin informacion' else 'sin informacion'
    except Exception as e:
        print("error en: ",importador," con el error", e)
        importador_h = 'Error'
    
    return importador_h

In [5]:
def homologar_exportador(exportador: str) -> str:
    """"""
    try:
        exportador_h = exportadores_dict[exportador] if exportador != 'sin informacion' else 'sin informacion'
    except Exception as e:
        print("error en: ",exportador," con el error", e)
        exportador_h = 'Error'
    
    return exportador_h

In [6]:
def normalizar(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula y quita espacios al principio o fin de la cadena"""
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        text_normalized = text.translate(trans).lower().strip()
    except:
        print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized

### funciones de clasificacion

In [7]:
def calc_valor_unitario(valor_total, unidades):
  """calcular valor_unitario dividiendo valor_total entre el número de unidades(previamente encontrado en descripcion comercial)
     return valor_unitario"""
  pass

def comprobar():
  """
    calc_valor_unitario(valor_total, unidades)
    este nuevo valor_unitario se debe contrastar contra el valor_unitario de medicamentos de la misma marca para
    verificar que no sea un outlier:
      TipoI = producto_terminado
      si es un outlier se clasificará como:
        TipoII = EstandarReferencia
      si no es outlier
        TipoII = Valor comercial
  """
  pass

def buscar_pres_medi_en_desc_comer():
  """buscar presentacion de medicamento en descripcion comercial:
      buscar unidades, tipo_unidad, concentracion, unidad_de_concentracion 
      en la descripcion_comercial.
      
      return unidad, tipo_unidad, concentracion, unidad_de_concentracion(25,pzas,50,mg) """
  pass 

def comprobar_principio_activo():
  """
  buscar_pres_medi_en_desc_comer()
  si estas están en su descripcion_comercial entonces no son PrincipioActivo AÚN
    return False
  si estas NO están en su descripcion_comercial:
    entonces podemos clasificarlas como PrincipioActivo
    return True
  """
  pass

def clasificar(row):
    """
    recibe cada fila del dataframe en su columna "descripcion_comercial",
    tratando de tipificarla:
    1)
    si estandar de referencia en descripcion_comercial
      tipo II = estander de referencia
    si unidad es igual a KG,G,L,MLL
        sí comprobar_principio_activo():
          tipo I = principio activo
        sí no:
          2) comrpobar_si_ER_o_VC()
    2)
         
    """
    pass
    

In [7]:
def clasify_er_t2_by_keywords(desc_comer : str):
    """looks for keywords in descripcion_comercial column, if the keywords are
    contained is clasify as estandar de referencia(er) on its tipoII"""
    if ("estandar de referencia" in desc_comer) or ("sustancia de referencia" in desc_comer) or ("reference standard" in desc_comer) or ("estandares de refrencia" in desc_comer): 
        return "ER"
    else:
        return None

In [8]:
def clasify_pa_t1_by_tipounidad(t_u : str):
    """looks in its tipo_unidad columns if g,kg,l,ml are contained
    it will be clasified as principio_activo(PA) on its tipoI"""
    if ('G'==t_u) or ('KG'==t_u) or ('L'==t_u) or ('ML'==t_u):
        return 'PA'
    else:
        return None

### funcion de llenado de datos

In [ ]:
[ 'Partida Aduanera', 
   'Fecha', 
   'Importador',
   'Exportador',
   'cantidad - Unidades',
   'unidad - tipo_unidad',
   'valor_total',
   'valor_unitario',
   'via',
   'descripcion_comercial', 
   'pais_compra_h',
   'pais_origen_h',
   'regimen_h',
   'nombre_y_marca',
   'importador_h',
   'exportador_h',
   'exportador_h_po',
   'exportador_h_pc',
   ______falta_____,
   concentracion,
   unidad_concentracion,
   cnis,
   status_medicamento,
   presentacion,
   forma_farma,
   nombre_medicamento,
   marca,
   empresa,
   unidosis,
   tipoI,
   tipoII
]

In [161]:
#MedicamentosDB['FORMA FARMA'].value_counts() #UNIDAD CONCENTRACION(mg,µG)	FORMA FARMA(Solución Inyectable,Comprimidos, Tabletas, Capsulas, Solución oral ) 
#caja, frasco, ampula, frasco ampula
UNIDAD_CONCENTRACION = ['mg','µG']
FORMA_FARMA = ['tabletas', 'capsulas','comprimidos','solucion inyectable', 'solucion oral']

In [9]:
def get_concentracion_from_desc_comer(row):
    """ """
    desc_comer = row['descripcion_comercial']
    #afinitor 10 mg  30 comprimidos (everolimus)
    expresion_1 = '\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral)'
    #giotrif 30mg (afatinib) 30 tabletas (medicamento de uso humano)
    expresion_1_1 = '\d+\.?\d*\s*mg.{0,20}\d{1,4}\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral)' #menos exacta que la expresion_1
    #prycel (dasatinib) frasco con  60 tabletas de 70 mg (medicamento de uso humano)
    expresion_2 = '\d{1,3}\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral).{0,5}\d+\.?\d*\s*mg'
    #avastin  100 mg/4 ml solucion inyectable (bevacizumab)
    expresion_soluciones = '\d+\.?\d*\s*mg.{0,30}\s*(solucion inyectable|solucion oral)'
    #mvasi (bevacizumab) ampula con 100 mg/ 4.0 ml (medicamento de uso humano)
    expresion_4 = '(frasco ampula|vial|ampula|jeringa)\s*.{0,5}\d+\s*mg'
    #kadcyla 100mg frasco ampula (trastuzumab emtansina)
    expresion_6 = '\d+\.?\d*\s*mg\s*(frasco ampula|vial|ampula|jeringa)'
    #tarceva 100 mg comprimidos  (erlotinib)
    expresion_3 = '\d+\.?\d*\s*mg\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral)'
    #kyprolis 60mg (carfilzomib) (medicamento de uso humano)
    expresion_5 = '\d+\.?\d*\s*mg'
    if not isinstance(desc_comer, str):
        return ['np.nan', 'np.nan','np.nan', 'np.nan']

    match = re.search(expresion_1, desc_comer)
    if match:#10 mg  30 comprimidos
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[0]
        unidad_concentracion = 'mg'
        presentacion = numeros[1]
        forma_farma = re.search("[a-z]{3,15}", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_1_1, desc_comer)
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[0]
        unidad_concentracion = 'mg'
        presentacion = numeros[1]
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_2, desc_comer) 
    if match:#60 capsulas de 250mg
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[1]
        unidad_concentracion = 'mg'
        presentacion = numeros[0]
        forma_farma = re.search("[a-z]{3,15}", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_3, desc_comer)
    if match:#250 mg tabletas
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[0]
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = re.search("[a-z]{3,15}", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_soluciones, desc_comer)
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    match = re.search(expresion_4, desc_comer)
    if match:#ampula con 100 mg
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = re.search("mg", info).group()
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|frasco|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    match = re.search(expresion_6, desc_comer)
    if match:#100mg frasco ampula
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = re.search("mg", info).group()
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|frasco|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma]

    #match = re.search(expresion_5, desc_comer)
    #if match:
    #    info = match.group()
    #    concentracion =  re.search("\d+", info).group()
    #    unidad_concentracion = 'mg'
    #    presentacion = 'sp'
    #    forma_farma = 'sff'
    #    return [concentracion, unidad_concentracion, presentacion, forma_farma]
    
    return [np.nan, np.nan, np.nan, np.nan]

### llenado de datos con NM

In [7]:
def fill_concentracion_issues_w_NM(import_row_nm):
    df_dict = MedicamentosDB[['CONCENTRACION','UNIDAD CONCENTRACION','PRESENTACION','FORMA FARMA','CNIS', 'STATUS', 'EMPRESA','UNIDOSIS']][ (MedicamentosDB['MEDICAMENTO'] ==import_row_nm['nombre']) & (MedicamentosDB['NOMBRE COMERCIAL']==import_row_nm['marca']) ].to_dict('records')
    
    if len(df_dict) == 0:
        print(f"sin match para{import_row_nm['nombre']}, {import_row_nm['marca']}")

    coincidencias = 0
    matches = 0
    empresa = ''
    concentracion = 0
    unidad_concentracion = ''
    presentacion = 0
    forma_farma = ''
    cnis = ''
    status_medicamento = ''
    c_r_c_llenado_datos = ''

    def set_values(p_empresa, p_concentracion, p_unidad_concentracion, p_presentacion, p_forma_farma, p_cnis, p_status_medicamento):#, msj):
        """setea los valores "globales" que se asignan despues del for principal """
        nonlocal empresa, concentracion, unidad_concentracion, presentacion, forma_farma, cnis, status_medicamento
        empresa = p_empresa
        concentracion = p_concentracion
        unidad_concentracion = p_unidad_concentracion
        presentacion = p_presentacion
        forma_farma = p_forma_farma 
        cnis = p_cnis
        status_medicamento = p_status_medicamento
        #c_r_c_llenado_datos = msj

    def check_if_update_values(p_empresa, p_concentracion, p_unidad_concentracion, p_presentacion, p_forma_farma, p_cnis, p_status_medicamento):
        """revisa los datos del match anterior y dependiendo el caso determina
        si actualizarlos o no"""
        if (isinstance(concentracion,str) or isinstance(p_concentracion,str)):
            c_r_c_llenado_datos = f"fallo en: {concentracion} o en: {p_concentracion}"
            return [
                np.nan,
                np.nan, 
                np.nan, 
                np.nan, 
                np.nan, 
                np.nan, 
                np.nan, 
                c_r_c_llenado_datos
            ]

        if p_concentracion > concentracion and p_presentacion > presentacion:
            set_values(p_empresa, p_concentracion, p_unidad_concentracion, p_presentacion, p_forma_farma, p_cnis, p_status_medicamento)
        elif p_concentracion > concentracion:
            set_values(p_empresa, p_concentracion, p_unidad_concentracion, p_presentacion, p_forma_farma, p_cnis, p_status_medicamento)
        elif concentracion > p_concentracion:
            pass
        elif p_presentacion > presentacion:
            set_values(p_empresa, p_concentracion, p_unidad_concentracion, p_presentacion, p_forma_farma, p_cnis, p_status_medicamento)

    for medicine_row in df_dict:
        contador = 0
        
        if str(medicine_row['CONCENTRACION']) in import_row_nm['descripcion_comercial']:
            contador += 1
        if medicine_row['UNIDAD CONCENTRACION'] in import_row_nm['descripcion_comercial']:
            contador += 1
        if str(medicine_row['PRESENTACION']) in import_row_nm['descripcion_comercial']:
            contador += 1
        if medicine_row['FORMA FARMA'] in import_row_nm['descripcion_comercial']:
            contador += 1
        
        #si hay coincidencias y es mayor en numero de estas que el match anterior(en caso de que halla)
        if contador > coincidencias:
            coincidencias = contador
            matches = 1 
            set_values(medicine_row['EMPRESA'],medicine_row['CONCENTRACION'],medicine_row['UNIDAD CONCENTRACION'],
                       medicine_row['PRESENTACION'], medicine_row['FORMA FARMA'], medicine_row['CNIS'], medicine_row['STATUS']) 
        
        #si existen las mismas coincidencias que el match anterior 
        elif contador > 0 and contador == coincidencias:
            matches += 1
            check_if_update_values(medicine_row['EMPRESA'],medicine_row['CONCENTRACION'],medicine_row['UNIDAD CONCENTRACION'],
                                medicine_row['PRESENTACION'], medicine_row['FORMA FARMA'], medicine_row['CNIS'], 
                                medicine_row['STATUS']) 
    c_r_c_llenado_datos = f"{coincidencias} coincidencias y {matches} matches"
    if coincidencias >= 3:
        return [
            empresa, 
            concentracion, 
            unidad_concentracion, 
            presentacion, 
            forma_farma, 
            cnis, 
            status_medicamento, 
            c_r_c_llenado_datos
        ]
    else:
        return [
            np.nan,
            np.nan, 
            np.nan, 
            np.nan, 
            np.nan, 
            np.nan, 
            np.nan, 
            c_r_c_llenado_datos
        ]
    #print(f"{coincidencias} coincidencias en {matches} matches para: {import_row_nm['nombre']}, {import_row_nm['marca']} en: {import_row_nm['descripcion_comercial']}")

## Normalizando Importadores y Exportadores

In [10]:
imports['Importador'].fillna('sin informacion', inplace=True)
imports['Exportador'].fillna('sin informacion', inplace=True)
imports['importador_h'] = imports['Importador'].apply(homologar_importador)
imports['exportador_h'] = imports['Exportador'].apply(homologar_exportador)
#se homologa porque se me olvidó homologar antes los exportadores e importadores y al concatenarlos con los paises que vienenen normalizados se veian tal que asi: AZTRAZENECA mexico
imports['importador_h'] = imports['importador_h'].apply(normalizar)
imports['exportador_h'] = imports['exportador_h'].apply(normalizar)
#se concatena el exportador mas el pais de origen y compra respectivamente
imports['exportador_h_po'] = imports.apply(lambda row: row['exportador_h'] + ' ' + row['pais_origen_h'] if row['exportador_h'] != 'sin informacion' else 'sin informacion',axis=1)
imports['exportador_h_pc'] = imports.apply(lambda row: row['exportador_h'] + ' ' + row['pais_compra_h'] if row['exportador_h'] != 'sin informacion' else 'sin informacion',axis=1)

## Normalizando Medicamentos

In [11]:
MedicamentosDB['MEDICAMENTO'] = MedicamentosDB['MEDICAMENTO'].apply(normalizar)
MedicamentosDB['NOMBRE COMERCIAL'] = MedicamentosDB['NOMBRE COMERCIAL'].apply(normalizar)
MedicamentosDB['FORMA FARMA'] = MedicamentosDB['FORMA FARMA'].apply(normalizar)

## llenado de datos con NM

In [12]:
#creacion de columnas para ir detectando cambios y más
imports['concentracion'] = ''
imports['unidad_concentracion'] = ''
imports['cnis'] = ''
imports['status_medicamento'] = ''
imports['presentacion'] = ''
imports['forma_farma'] = ''
imports['nombre_medicamento'] = ''
imports['empresa'] = ''
imports['unidosis'] = ''
imports['tipoI'] = ''
imports['tipoII'] = ''
imports['c_r_c_llenado_datos'] = ''

In [ ]:
#llena los campos descritos con "fill_concentracion_issues_w_NM"
imports[['empresa','concentracion','unidad_concentracion','presentacion','forma_farma','cnis','status_medicamento', 'c_r_c_llenado_datos']]=imports[imports['nombre_y_marca']=='NM'].apply(fill_concentracion_issues_w_NM, axis=1, result_type='expand')

In [56]:
imports[['Exportador','empresa','exportador_h','descripcion_comercial','concentracion','unidad_concentracion','presentacion','forma_farma','cnis','status_medicamento', 'c_r_c_llenado_datos']][imports['nombre_y_marca']=='NM'].to_csv('prueba.csv')

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE y MARCA
MedicamentosDB[['CONCENTRACION','UNIDAD CONCENTRACION','PRESENTACION','FORMA FARMA']][ (MedicamentosDB['MEDICAMENTO'] == 'trastuzumab' ) & (MedicamentosDB['NOMBRE COMERCIAL']=='trazimera') ]

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE 
MedicamentosDB[['CONCENTRACION','UNIDAD CONCENTRACION','PRESENTACION','FORMA FARMA']][ MedicamentosDB['MEDICAMENTO'] == 'ribociclib' ]

## Clasificación tipo 1

In [13]:
imports['tipoII'] = imports['descripcion_comercial'].apply(clasify_er_t2_by_keywords)
imports['tipoI'] = imports['tipo_unidad'].apply(clasify_pa_t1_by_tipounidad)

In [ ]:
unidades = {
'PZA':'pieza',
'CAJ':'caja',
'BOT':'botella',
'JGO':'juego',
'L':'litro',
'MLL':'mililitro',
'G':'gramos',      
'KG':'kilogramos'
}
imports['tipo_unidad'].value_counts(dropna=False)

In [ ]:
imports['descripcion_comercial'][imports['descripcion_comercial'].str.contains("vial")]

In [65]:
estandar_referecia_keyWords = ['estandar', 'referencia','sustancia de referencia']#puede ser "substancia solucion"

In [ ]:
imports[['descripcion_comercial','cantidad','unidad', 'valor_total','valor_unitario','regimen_h','nombre_y_marca']][(imports['unidad']=='G') & (~imports['descripcion_comercial'].str.contains("estandar de referencia"))]

## LLenado de presentación farmacologica

In [14]:
imports[['concentracion','unidad_concentracion','presentacion','forma_farma']] = imports.apply(get_concentracion_from_desc_comer, axis=1, result_type='expand')

In [16]:
#full imports selected columns
imports[['tipo_unidad','unidades','descripcion_comercial','nombre','marca','concentracion','unidad_concentracion','presentacion','forma_farma','tipoI','tipoII']].to_csv('coercion.csv')

In [15]:
#imports COMPLETE only with concentracion
imports[~imports['concentracion'].isna()].to_csv('cp_complete.csv')

In [17]:
#imports only with concentracion
imports[['unidades','tipo_unidad','nombre_y_marca','nombre','marca','descripcion_comercial','presentacion','forma_farma','concentracion','unidad_concentracion','tipoI','tipoII']][~imports['concentracion'].isna()].to_csv('cp.csv')

In [18]:
#imports only without concentracion
imports[['unidades','tipo_unidad','nombre_y_marca','nombre','marca','descripcion_comercial','presentacion','forma_farma','concentracion','unidad_concentracion','tipoI','tipoII']][imports['concentracion'].isna()].to_csv('sp.csv')

In [ ]:
#sin sp sin sff y sin ningun dato vacio
imports[['forma_farma','concentracion','unidad_concentracion','presentacion']][(imports['presentacion']!='sp') & (imports['forma_farma']!='sff')&(imports['nombre_y_marca']=='NM')].dropna(how='any')

In [26]:
imports[imports['concentracion'].isna()].shape  #98

(3187, 32)